### Capstone Project 2 Data Cleaning/Wrangling

In [1]:
import pandas as pd
import numpy as np

First, I'll create a dataframe from the dataset, which is in three CSV files due to GitHub's file size limit. I specify latin1 encoding because there was at least one byte that could not be interpreted using utf-8. (Some) missing values are encoded with "__NA__" in this dataset.

In [2]:
df1 = pd.read_csv('CCES2016.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df2 = pd.read_csv('CCES2.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])
df3 = pd.read_csv('CCES3.csv', encoding = 'latin1', index_col = 'V101', na_values = ['__NA__'])

#'utf-8' codec can't decode byte 0xe1 in position 5: unexpected end of data

df = df1.append(df2)
df = df.append(df3)

df = df.reset_index()

C:\Users\joshy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (50,51,52,53,54,55,79,119,254,262,270,271,274,286,296,298,300,302,304,306,308,310,314,316,318,322,326,328,332,333,370,410,411,412,413,414,415,452,458,460,462,494,535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\joshy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (46,47,48,49,50,51,52,53,54,55,79,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,164,165,166,167,168,169,170,199,200,201,203,228,239,240,242,243,254,262,264,270,271,274,278,279,286,296,298,300,302,304,306,308,310,312,314,316,317,318,320,322,324,326,328,330,332,334,370,406,407,408,409,410,411,412,413,414,415,443,450,452,458,460,520,535) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=resu

In [3]:
df.head()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,tookpost,lookupzip,inputstate,cdid113,...,CL_E2016PPEP,CL_E2016PPVM,CL_E2016PEP,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname
0,222168628,1.344214,1.042267,1.195346,0.944967,0.730450,Yes,3442,New Hampshire,2,...,NaN,NaN,NaN,NaN,05:44.9,36:38.8,24:32.9,40:18.8,NaN,NaN
1,273691199,1.182352,1.280103,1.280565,0.767256,0.892838,Yes,70118,Louisiana,2,...,NaN,NaN,NaN,NaN,54:58.5,20:41.7,02:55.1,09:26.9,622.0,NEW ORLEANS
2,284214415,0.217396,NaN,0.248930,0.090604,0.000000,No,63851,Missouri,8,...,NaN,NaN,NaN,NaN,54:12.8,24:44.8,NaN,NaN,632.0,PADUCAH-CAPE GIRARD-HARSBG
3,287557695,0.532320,NaN,0.928750,0.142583,0.000000,No,36703,Alabama,7,...,NaN,NaN,NaN,NaN,06:21.3,26:00.1,NaN,NaN,698.0,MONTGOMERY-SELMA
4,290387662,1.261106,1.092519,1.161894,1.206498,1.019007,Yes,80526,Colorado,2,...,NaN,NaN,NaN,NaN,56:31.1,14:12.2,09:28.0,21:05.9,751.0,DENVER


The birth year ("birthyr") column is missing in the CSV file containing the 2016 CCES data, for some reason. Fortunately, I was able to find this column in another spreadsheet that showed the cumulative CCES results from 2006 to 2017, which uses the same case ID index. The cumuative16.csv file contains the rows from that spreadsheet for the 2016 survey.

In [4]:
cumulative = pd.read_csv('cumulative16.csv', encoding = 'latin1')
cumulative = cumulative[['case_id', 'birthyr']]
cumulative.head()

,case_id,birthyr
0,222168628,1969
1,273691199,1994
2,284214415,1964
3,287557695,1988
4,290387662,1982


Now, I merge the two dataframes, using the names of the case ID column for each dataframe.

In [5]:
df = pd.merge(df, cumulative, left_on = 'V101', right_on = 'case_id')

In [7]:
df['birthyr'] = df['birthyr_y']

In [8]:
df.drop(['birthyr_y'], axis = 1)
df[['V101', 'birthyr']].head()

,V101,birthyr
0,222168628,1969
1,273691199,1994
2,284214415,1964
3,287557695,1988
4,290387662,1982


In [9]:
#ensure that case IDs and birth years match for both dataframes
different_index = []
for i in range(64600):
    if df['V101'].iloc[i] != cumulative['case_id'].iloc[i]:
        different_index.append(i)
        
different_year = []
for i in range(64600):
    if df['birthyr'].iloc[i] != cumulative['birthyr'].iloc[i]:
        different_year.append((i, df['birthyr'].iloc[i], cumulative['birthyr'].iloc[i] ))
        
print(len(different_index))
print(len(different_year))

0
0


### Column selection and Missing Values

Now, I have the data imported into a dataframe. There are a large number of columms, many of which are not likely to be useful for this project. First, I'll print an alphabetical list of columns:

In [10]:
columns_indices = []

for i in range(len(df.columns)):
    columns_indices.append((i, df.columns[i]))
    
columns_indices = sorted(columns_indices, key = lambda x: x[1].lower())

for item in columns_indices:
    if item[0] < 10:
        print(str(item[0]) + "   " + str(item[1]))
    elif item[0] < 100:
        print(str(item[0]) + "  " + str(item[1]))
    else:
        print(str(item[0]) + " " + str(item[1]))

70  add_confirm
104 Asian_origin_1
113 Asian_origin_10
114 Asian_origin_11
115 Asian_origin_12
116 Asian_origin_13
117 Asian_origin_14
118 Asian_origin_15
105 Asian_origin_2
106 Asian_origin_3
107 Asian_origin_4
108 Asian_origin_5
109 Asian_origin_6
110 Asian_origin_7
111 Asian_origin_8
112 Asian_origin_9
119 Asian_origin_t
372 AskParty_post
565 birthyr
75  birthyr_x
564 birthyr_y
563 case_id
121 CC16_300_1
122 CC16_300_2
123 CC16_300_3
124 CC16_300_4
125 CC16_300_5
126 CC16_300_6
127 CC16_300b
128 CC16_300c
129 CC16_300d_1
130 CC16_300d_2
131 CC16_300d_3
132 CC16_300d_4
133 CC16_300d_5
134 CC16_301a
135 CC16_301b
136 CC16_301c
137 CC16_301d
138 CC16_301e
139 CC16_301f
140 CC16_301g
141 CC16_301h
142 CC16_301i
143 CC16_301j
144 CC16_301k
145 CC16_301l
146 CC16_301m
147 CC16_301n
148 CC16_301o
149 CC16_302
150 CC16_303
151 CC16_304
152 CC16_305_1
161 CC16_305_10
162 CC16_305_11
153 CC16_305_2
154 CC16_305_3
155 CC16_305_4
156 CC16_305_5
157 CC16_305_6
158 CC16_305_7
159 CC16_305_8
160 C

Next, I look at how many missing values there are by column:

In [1]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1])

for i in range(1, len(nan_counts)):
    print(nan_counts[i][0], nan_counts[i][1])

NameError: name 'df' is not defined

215 variables have fewer than 10% of the data missing.

In [12]:
df.groupby(['tookpost']).count()

,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,lookupzip,inputstate,cdid113,cdid115,...,CL_E2016PVM,starttime_pre,endtime_pre,starttime_post,endtime_post,dma,dmaname,case_id,birthyr_y,birthyr
tookpost,,,,,,,,,,,,,,,,,,,,,
No,11701,11701,43,4280,11701,11701,11701,11701,11701,11701,...,576,11701,11701,0,0,10827,10827,11701,11701,11701
Yes,52899,52899,52899,52868,52899,52899,52899,52899,52899,52899,...,7742,52899,52899,52899,52899,48706,48706,52899,52899,52899


52899 took the post survey.

Because the number of variables was so large, I created a spreadsheet to keep track of notes for each variable and decide whether it should be included in this project. Here I import and extract data from this spreadsheet.

In [13]:
vars_df = pd.read_csv('variable_list.csv', encoding = 'latin1')
variables = vars_df.loc[vars_df['Include'] == 'yes']['Name'].tolist()

In [14]:
print(variables)

['tookpost', 'birthyr', 'gender', 'educ', 'race', 'CC16_331_1', 'CC16_331_2', 'CC16_331_3', 'CC16_331_7', 'employ', 'milstat_1', 'milstat_2', 'milstat_3', 'milstat_4', 'CC16_351K', 'faminc', 'CC16_334a', 'CC16_334b', 'pew_religimp', 'ideo5', 'CC16_334d', 'pew_bornagain', 'CC16_334c', 'CC16_333d', 'marstat', 'CC16_333b', 'CC16_333a', 'CC16_332a', 'CC16_332d', 'CC16_332e', 'CC16_333c', 'religpew', 'ownhome', 'CC16_332c', 'CC16_332f', 'CC16_351I', 'CC16_332b', 'child18', 'CC16_351B', 'CC16_351E', 'union', 'CC16_327', 'immstat', 'CC16_351G', 'CC16_351F', 'CC16_330a', 'CC16_351H', 'CC16_330e', 'CC16_330b', 'unionhh', 'CC16_330d', 'CC16_335', 'hispanic', 'sexuality', 'investor', 'trans', 'inputstate_post', 'votereg_post', 'CC16_414_1', 'CC16_414_2', 'CC16_414_3', 'CC16_414_4', 'CC16_414_5', 'CC16_414_6', 'CC16_417a_1', 'CC16_417a_2', 'CC16_417a_3', 'CC16_417a_4', 'CC16_417a_5', 'CC16_421a', 'CC16_422d', 'CC16_422c', 'edloan', 'CC16_422f', 'CC16_422e', 'CC16_426_1', 'CC16_426_2', 'CC16_426_3'

In [15]:
df = df[variables]

I am interested in people who took the post-election survey, indicating how they voted.

In [16]:
df = df[df['tookpost'] == 'Yes']

#df.info()

Now, let's look at how many missing values are remaining after removing most of the columns:

In [17]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(1, len(nan_counts)):
    print(nan_counts[i])

('CC16_410a', 7657)
('hispanic', 3903)
('CC16_401', 3814)
('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)
('CC16_330a', 186)
('CC16_418a', 176)
('CC16_426_5', 172)
('CC16_351F', 168)
('CC16_426_3', 167)
('CC16_426_2', 145)
('CC16_426_1', 139)
('immstat', 127)
('CC16_422e', 124)
('CC16_422f', 121)
('investor', 119)
('CC16_351E', 118)
('CC16_351B', 111)
('edloan', 110)
('CC16_327', 108)
('child18', 96)
('union', 87)
('CC16_351I', 82)
('CC16_332b', 78)
('CC16_332f', 75)
('CC16_332c', 66)
('CC16_422c', 65)
('trans', 62)
('CC16_422d', 62)
('ownhome', 61)
('CC16_333c', 55)
('CC16_332a', 53)
('CC16_332e', 53)
('CC16_333a', 52)
('CC16_334c', 49)
('CC16_333b', 48)
('CC16_332d', 48)
('religpew', 46)
('CC16_333d', 44)
('CC16_334d', 39)
('marstat', 39)
('sexuality', 31)
('CC16_334b', 30)
('pew_bornagain', 29)
('CC16_334a', 28)
('pew_religimp', 24)
('ideo5', 19)
('CC16_351K', 16)
('faminc', 12)
('C

CC16_326 (2012 pres vote) had the most missing data when just considering post survey. Given how many there are, I will just exclude rather than try to impute it.

In [18]:
df = df.drop(['CC16_326'], axis = 1)

The variable with the next-largest number of missing rows is CC16_410a (Who did you vote for in 2016?). This is an essential outcome variable so I cannot exclude it. Fortunately, the missing rows come from respondents who did not answer this question because they already indicated that they did not vote for question CC16_401.

In [19]:
df.groupby(['CC16_401']).size()

CC16_401
I attempted to vote but did not or could not.      487
I definitely voted in the General Election.      45292
I did not vote in the election this November.     1802
I thought about voting this time - but didn't      750
I usually vote, but didn't this time.              754
dtype: int64

I create a column encoding whether the respondent voted:

In [20]:
def voted(s):
    if "definitely voted" in str(s):
        return 1
    else:
        return 0

df['voted'] = df['CC16_401'].apply(voted)

In [21]:
df.groupby(['voted']).size()

voted
0     7607
1    45292
dtype: int64

In [22]:
df.groupby(['CC16_410a']).size()

CC16_410a
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election       81
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

Now I populate the CC16_410a column based on the voted column:

In [23]:
df.loc[df['voted'] == 0, 'CC16_410a'] = 'I didn\'t vote in this election'

In [24]:
df.groupby(['CC16_410a']).size()

CC16_410a
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election     7688
I'm not sure                        229
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [25]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse=True)

for i in range(0, 20):
    print(nan_counts[i])

('hispanic', 3903)
('CC16_401', 3814)
('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)


CC16_410a (presidential vote) is now almost entirely filled out. CC16_401 itself has a lot of missing data due to respondents not being asked that question, but the information is contained in CC16_410a anyway, so I won't worry about it.

In [26]:
df = df.drop(['CC16_401'], axis = 1)

The next column is the hispanic column. Again, the missing values come from respondents not being asked the question, and I can get this data from the race column (if respondents already answered "Hispanic" when asked their race, they were not asked again whether they were Hispanic). 

In [27]:
df.groupby(['race']).size()

race
Asian               1739
Black               4931
Hispanic            3742
Middle Eastern        99
Mixed               1169
Native American      440
Other                680
White              40099
dtype: int64

In [28]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     1607
dtype: int64

In [29]:
df.loc[df['race'] == 'Hispanic', 'hispanic'] = 'Yes'

In [30]:
df.groupby(['hispanic']).size()

hispanic
No     47389
Yes     5349
dtype: int64

In [31]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse = True)

for i in range(0, 10):
    print(nan_counts[i])

('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('CC16_351H', 232)
('CC16_426_4', 207)
('CC16_351G', 188)
('CC16_330a', 186)
('CC16_418a', 176)


For the other columns, missing values come from respondents skipping the question, and the number of missing values is relatively small.

### Implicit missing values

However, there are still missing values that are not coded as NaN yet, because some answers represent the respondent declining to provide any information. I will now print the unique values for each column to check for this:

In [32]:
for col in df.columns.sort_values():
    print (col, df[col].unique())

CC16_327 ["No, didn't vote in a primary or caucus"
 'Yes, voted in a primary or caucus' nan]
CC16_330a ['Support' 'Oppose' nan]
CC16_330b ['Oppose' 'Support' nan]
CC16_330d ['Oppose' 'Support' nan]
CC16_330e ['Oppose' 'Support' nan]
CC16_331_1 ['No' 'Yes']
CC16_331_2 ['Yes' 'No']
CC16_331_3 ['No' 'Yes']
CC16_331_7 ['Yes' 'No']
CC16_332a ['Support' 'Oppose' nan]
CC16_332b ['Oppose' 'Support' nan]
CC16_332c ['Support' 'Oppose' nan]
CC16_332d ['Oppose' 'Support' nan]
CC16_332e ['Oppose' 'Support' nan]
CC16_332f ['Oppose' 'Support' nan]
CC16_333a ['Oppose' 'Support' nan]
CC16_333b ['Oppose' 'Support' nan]
CC16_333c ['Oppose' 'Support' nan]
CC16_333d ['Oppose' 'Support' nan]
CC16_334a ['Support' 'Oppose' nan]
CC16_334b ['Support' 'Oppose' nan]
CC16_334c ['Oppose' 'Support' nan]
CC16_334d ['Support' 'Oppose' nan]
CC16_335 ['Favor' 'Oppose' nan]
CC16_351B ['For' 'Against' nan]
CC16_351E ['For' 'Against' nan]
CC16_351F ['Against' 'For' nan]
CC16_351G ['For' 'Against' nan]
CC16_351H ['Against' 

Family income (faminc), sexuality, trans have a "prefer not to say" option. Ideology (ideo5), presidential vote (CC16_410a), has a "Not sure" option, which is arguably missing information. Voter registration (votereg_post) has a "don't know" option.

In [33]:
df.groupby(['ideo5']).size()

ideo5
Conservative         12183
Liberal              10147
Moderate             17876
Not sure              3348
Very conservative     4428
Very liberal          4898
dtype: int64

In [34]:
df['ideo5'] = df['ideo5'].replace('Not sure', 'Moderate or Not sure')
df['ideo5'] = df['ideo5'].replace('Moderate', 'Moderate or Not sure')

In [35]:
df.groupby(['ideo5']).size()

ideo5
Conservative            12183
Liberal                 10147
Moderate or Not sure    21224
Very conservative        4428
Very liberal             4898
dtype: int64

I'm deciding to collapse the "not sure" and "moderate" categories on the grounds that someone who is undecided about their ideology has a similar ideology to someone who does not subscribe to one side or the other.

Next, I will replace the "I'm not sure" answer to the presidential vote question, and the voter registration question, with NaN.

In [36]:
df['CC16_410a'] = df['CC16_410a'].replace("I'm not sure", np.NaN)
df.groupby(['CC16_410a']).size()

CC16_410a
Donald Trump (Republican)         18755
Evan McMullin (Independent)         163
Gary Johnson (Libertarian)         1829
Hillary Clinton (Democrat)        22136
I didn't vote in this election     7688
Jill Stein (Green)                  913
Other                              1136
dtype: int64

In [37]:
df.groupby(['votereg_post']).size()

votereg_post
Don't know      279
No             3735
Yes           48885
dtype: int64

In [38]:
df['votereg_post'] = df['votereg_post'].replace("Don't know", np.NaN)
df.groupby(['votereg_post']).size()

votereg_post
No      3735
Yes    48885
dtype: int64

I do the same thing with the sexuality and trans columns:

In [39]:
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    47571
Lesbian / gay woman          637
Other                        485
Prefer not to say           1074
dtype: int64

In [40]:
df['sexuality'] = df['sexuality'].replace('Prefer not to say', np.NaN)
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    47571
Lesbian / gay woman          637
Other                        485
dtype: int64

In [41]:
df.groupby(['trans']).size()

trans
No                   51319
Prefer not to say      649
Yes                    869
dtype: int64

In [42]:
df['trans'] = df['trans'].replace('Prefer not to say', np.NaN)
df.groupby(['trans']).size()

trans
No     51319
Yes      869
dtype: int64

In [43]:
nan_counts = []
for col in df.columns:
    nan_counts.append((col, df[col].isnull().sum()))
    
nan_counts = sorted(nan_counts, key = lambda x: x[1], reverse = True)

for i in range(0, 10):
    print(nan_counts[i])

('sexuality', 1105)
('trans', 711)
('CC16_335', 431)
('CC16_330d', 355)
('CC16_330b', 325)
('CC16_330e', 311)
('unionhh', 306)
('votereg_post', 279)
('CC16_410a', 279)
('CC16_351H', 232)


In [44]:
cleaned_df = df.dropna(axis = 0, how = 'any')
len(cleaned_df.index)

46823

After dropping missing values, there are now 46823 rows remaining, out of 52899 total respondents who responded to the post-survey. This seems like a large number of points being lost. Because a large majority of respondents reported being heterosexual, and a very large majority report not being trans, I will fill in the missing values with the mode for these columns.

In [45]:
df['sexuality'] = df['sexuality'].fillna('Heterosexual / straight')
df.groupby(['sexuality']).size()

sexuality
Bisexual                    1608
Gay man                     1493
Heterosexual / straight    48676
Lesbian / gay woman          637
Other                        485
dtype: int64

In [46]:
df['trans'] = df['trans'].fillna('No')

df.groupby(['trans']).size()

trans
No     52030
Yes      869
dtype: int64

### Transforming values

Now there are a lot of variables with text values that need to be converted to binary or numeric variables.

In [47]:
df['age'] = 2018 - df['birthyr']
df = df.drop(['birthyr'], axis = 1)

Family income is a text variable indicating which range the respondent belongs to:

In [48]:
df.groupby(['faminc']).size()

faminc
$10,000 - $19,999      3543
$100,000 - $119,999    3442
$120,000 - $149,999    3021
$150,000 - $199,999    1642
$150,000 or more        136
$20,000 - $29,999      5074
$200,000 - $249,999     721
$250,000 - $349,999     380
$30,000 - $39,999      5361
$350,000 - $499,999     162
$40,000 - $49,999      4742
$50,000 - $59,999      4745
$500,000 or more        122
$60,000 - $69,999      3798
$70,000 - $79,999      3835
$80,000 - $99,999      4760
Less than $10,000      1911
Prefer not to say      5492
dtype: int64

In [49]:
df['faminc'].isnull().sum()

12

To convert family income, I will use the midpoint of the range, when a range is given. For the open ended (150,000 or more, 500,000 or more), I will use the bottom of that range. I am not sure why there was a 150,000 or more option when that overlaps with the other answers.

In [50]:
def faminc_to_int(s):
    #print(s)
    if pd.isnull(s):
        return np.NaN
    if "Prefer" in s:
        return np.NaN
    elif "Less" in s:
        return 5000
    elif "more" in s:
        incomes = int(s.replace('$', '').replace(',', '').split(' ')[0])
    else:
        incomes = [int(x) for x in s.replace('$', '').replace(' ', '').replace(',', '').split('-')]
    return int(np.mean(incomes))

faminc_to_int('$10,000 - $19,999')

14999

In [51]:
print(faminc_to_int('Less than $10,000'))
print(faminc_to_int('$500,000 or more'))

5000
500000


In [52]:
df['faminc_int'] = df['faminc'].apply(faminc_to_int)

In [53]:
df[['faminc', 'faminc_int']][0:10]

,faminc,faminc_int
0,Prefer not to say,NaN
1,"$50,000 - $59,999",54999.0
4,"$60,000 - $69,999",64999.0
5,"Less than $10,000",5000.0
6,"$20,000 - $29,999",24999.0
8,"$30,000 - $39,999",34999.0
9,"$60,000 - $69,999",64999.0
10,"$100,000 - $119,999",109999.0
11,"$40,000 - $49,999",44999.0
12,"$10,000 - $19,999",14999.0


In [54]:
df['faminc'] = df['faminc_int']
df.drop(['faminc_int'], axis = 1, inplace = True)

In [55]:
df.describe()

,faminc,voted,age
count,47395.000000,52899.000000,52899.000000
mean,69937.330394,0.856198,52.684285
std,57994.244039,0.350893,16.157814
min,5000.000000,0.000000,20.000000
25%,34999.000000,1.000000,39.000000
50%,54999.000000,1.000000,55.000000
75%,89999.000000,1.000000,65.000000
max,500000.000000,1.000000,97.000000


median of family income is 54999, I'll fill in the missing values accordingly.

In [56]:
df['faminc'] = df['faminc'].fillna(54999)

Now, look at which values exist in the remaining columns:

In [57]:
for col in df.columns:
    print (col, df[col].unique())

tookpost ['Yes']
gender ['Female' 'Male']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
CC16_331_1 ['No' 'Yes']
CC16_331_2 ['Yes' 'No']
CC16_331_3 ['No' 'Yes']
CC16_331_7 ['Yes' 'No']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 ['No' 'Yes']
milstat_2 ['No' 'Yes']
milstat_3 ['No' 'Yes']
milstat_4 ['Yes' 'No']
CC16_351K ['For' 'Against' nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
CC16_334a ['Support' 'Oppose' nan]
CC16_334b ['Support' 'Oppose' nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' n

Many binary variables have text answers, namely support/oppose, for/against, and yes/no. I convert these to 1 and 0.

In [58]:
df1 = df.replace({'Support': 1, 'Oppose' : 0, 'For': 1, 'Against': 0, 'Yes': 1, 'No': 0, 'Favor': 1})

In [59]:
for col in df1.columns:
    print (col, df[col].unique())
    print (col, df1[col].unique())

tookpost ['Yes']
tookpost [1]
gender ['Female' 'Male']
gender ['Female' 'Male']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
CC16_331_1 ['No' 'Yes']
CC16_331_1 [0 1]
CC16_331_2 ['Yes' 'No']
CC16_331_2 [1 0]
CC16_331_3 ['No' 'Yes']
CC16_331_3 [0 1]
CC16_331_7 ['Yes' 'No']
CC16_331_7 [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 ['No' 'Yes']
milstat_1 [0 1]
milstat_2 ['No' 'Yes']
milstat_2 [0 1]
milstat_3 ['No' 'Yes']
milstat_3 [0 1]
milstat_4 ['Yes' 

In [60]:
df = df1

For the five-point categorical answers, I will be converting these to dummy variables, so there is no need to do a value replacement. However, I will collapse the five answers into three answers to reduce the number of columns that are created. (may revisit this)

In [62]:
df = df.replace({'Greatly decrease': 'Decrease', 'Slightly decrease' : 'Decrease' , 'Slightly increase': 'Increase', 
                  'Greatly increase': 'Increase', 
                  'Strongly disagree': 'Disagree', 'Somewhat disagree': 'Disagree', 'Neither agree nor disagree': 'Neutral',
                 'Somewhat agree': 'Agree', 'Strongly agree': 'Agree'})

In [63]:
#list of questions with five categorical answers
fivepoint = ['CC16_422d', 'CC16_422c', 'CC16_422f', 'CC16_422e', 'CC16_426_1', 'CC16_426_2', 'CC16_426_3', 'CC16_426_5', 
             'CC16_426_4']

In [64]:
for col in fivepoint:
    df = df.merge(pd.get_dummies(df[col], prefix = col, drop_first = True), left_index = True, right_index = True)

In [65]:
df.columns

Index(['tookpost', 'gender', 'educ', 'race', 'CC16_331_1', 'CC16_331_2',
       'CC16_331_3', 'CC16_331_7', 'employ', 'milstat_1',
       ...
       'CC16_426_1_Increase', 'CC16_426_1_Maintain', 'CC16_426_2_Increase',
       'CC16_426_2_Maintain', 'CC16_426_3_Increase', 'CC16_426_3_Maintain',
       'CC16_426_5_Increase', 'CC16_426_5_Maintain', 'CC16_426_4_Increase',
       'CC16_426_4_Maintain'],
      dtype='object', length=101)

In [66]:
#rename variables
df['presvote'] = df['CC16_410a']
df.drop(['CC16_410a'], axis = 1, inplace = True)

Converting a couple more text values to 1 and 0:

In [67]:
df = df.replace({'Female': 0, 'Male' : 1})
df = df.replace({"No, didn\'t vote in a primary or caucus": 0, 'Yes, voted in a primary or caucus': 1})

In [68]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
CC16_331_1 [0 1]
CC16_331_2 [1 0]
CC16_331_3 [0 1]
CC16_331_7 [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
CC16_351K [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
CC16_334a [  1.   0.  nan]
CC16_334b [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
CC16_334d [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
CC16_334c [  0.   1.  nan]
CC16

Create binary variables based on whether the respondent has served in the military, and whether a family member has served.

In [69]:
df['veteran'] = ((df['milstat_1'] == 1) | (df['milstat_3'] == 1)).astype(int)
df['family_vet'] = ((df['milstat_2'] == 1) | (df['milstat_4'] == 1)).astype(int)

In [70]:
df[['veteran', 'family_vet']].head(5)

,veteran,family_vet
0,0,1
1,0,1
4,0,1
5,0,0
6,0,0


Create dummies for race:

In [72]:
df.groupby(['race']).size()

race
Asian               1739
Black               4931
Hispanic            3742
Middle Eastern        99
Mixed               1169
Native American      440
Other                680
White              40099
dtype: int64

In [73]:
df = df.merge(pd.get_dummies(df['race'], prefix = 'race', drop_first = True), left_index = True, right_index = True)

There is already a Hispanic variable (note that Hispanic isn't mutually exclusive with other races, so the two columns have different values). So I'll drop the dummy created for Hispanic.

In [74]:
df.drop(['race_Hispanic'], axis = 1, inplace = True)

Binary variable for 4-year college education or above.

In [75]:
df['college'] = ((df['educ'] == '4-year') | (df['educ'] == 'Post-grad')).astype(int)
df[['college', 'educ']].head(10)

,college,educ
0,0,High school graduate
1,0,High school graduate
4,1,4-year
5,0,High school graduate
6,0,High school graduate
8,0,High school graduate
9,0,High school graduate
10,1,4-year
11,0,Some college
12,0,Some college


Dummies for party ID:

In [76]:
df.groupby(['CC16_421a']).size()

CC16_421a
Democrat       20270
Independent    15787
Other           2595
Republican     14245
dtype: int64

In [77]:
df['PartyID'] = df['CC16_421a'].replace({'Other': 'Independent'})

In [78]:
df = df.merge(pd.get_dummies(df['PartyID'], prefix = 'Party', drop_first = True), left_index = True, right_index = True)

In [80]:
df[['CC16_421a', 'Party_Independent', 'Party_Republican']][:10]

,CC16_421a,Party_Independent,Party_Republican
0,Republican,0,1
1,Independent,1,0
4,Democrat,0,0
5,Democrat,0,0
6,Republican,0,1
8,Republican,0,1
9,Republican,0,1
10,Democrat,0,0
11,Independent,1,0
12,Other,1,0


Dummies for religion:

In [81]:
df = df.merge(pd.get_dummies(df['religpew'], prefix = 'relig', drop_first = True), left_index = True, right_index = True)

Convert sexuality to a binary variable- straight or not straight

In [82]:
df['straight'] = (df['sexuality'] == 'Heterosexual / straight').astype(int)

In [83]:
df.groupby(['immstat']).size()

immstat
First generation          4811
Immigrant Citizen         3121
Immigrant non-citizen      849
Second generation        11321
Third generation         32670
dtype: int64

In [84]:
df['immigrant'] = ((df['immstat'] == 'Immigrant Citizen') | (df['immstat'] == 'Immigrant non-citizen')).astype(int)

df[['immigrant', 'immstat']][:20]

,immigrant,immstat
0,0,Third generation
1,0,Third generation
4,0,Third generation
5,0,Third generation
6,0,Third generation
8,0,Third generation
9,0,Third generation
10,0,Second generation
11,0,Third generation
12,0,First generation


In [85]:
df.groupby(['union']).size()

union
I am not now, nor have I been, a member of a labor union    38907
I formerly was a member of a labor union                    10174
Yes, I am currently a member of a labor union                3731
dtype: int64

In [86]:
df['union_member'] = ((df['union'] == 'Yes, I am currently a member of a labor union') 
                      | (df['union'] == 'I formerly was a member of a labor union')).astype(int)

df[['union_member', 'union']][5:25]

,union_member,union
8,0,"I am not now, nor have I been, a member of a l..."
9,0,"I am not now, nor have I been, a member of a l..."
10,1,I formerly was a member of a labor union
11,0,"I am not now, nor have I been, a member of a l..."
12,0,"I am not now, nor have I been, a member of a l..."
14,0,"I am not now, nor have I been, a member of a l..."
15,0,"I am not now, nor have I been, a member of a l..."
16,0,"I am not now, nor have I been, a member of a l..."
17,0,"I am not now, nor have I been, a member of a l..."
18,0,"I am not now, nor have I been, a member of a l..."


In [87]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
CC16_331_1 [0 1]
CC16_331_2 [1 0]
CC16_331_3 [0 1]
CC16_331_7 [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
CC16_351K [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
CC16_334a [  1.   0.  nan]
CC16_334b [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
CC16_334d [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
CC16_334c [  0.   1.  nan]
CC16

In [88]:
df['presvote'] = df['presvote'].replace({'Donald Trump (Republican)': 'Trump', 'Hillary Clinton (Democrat)': 'Clinton', 
                 "I didn't vote in this election": 'None', 'Gary Johnson (Libertarian)': 'Johnson',
                'Evan McMullin (Independent)': 'McMullin', 'Jill Stein (Green)': 'Stein', })

In [89]:
df = df.merge(pd.get_dummies(df['presvote'], prefix = 'pres', drop_first = True), left_index = True, right_index = True)

In [90]:
for col in df.columns:
    print(col)

tookpost
gender
educ
race
CC16_331_1
CC16_331_2
CC16_331_3
CC16_331_7
employ
milstat_1
milstat_2
milstat_3
milstat_4
CC16_351K
faminc
CC16_334a
CC16_334b
pew_religimp
ideo5
CC16_334d
pew_bornagain
CC16_334c
CC16_333d
marstat
CC16_333b
CC16_333a
CC16_332a
CC16_332d
CC16_332e
CC16_333c
religpew
ownhome
CC16_332c
CC16_332f
CC16_351I
CC16_332b
child18
CC16_351B
CC16_351E
union
CC16_327
immstat
CC16_351G
CC16_351F
CC16_330a
CC16_351H
CC16_330e
CC16_330b
unionhh
CC16_330d
CC16_335
hispanic
sexuality
investor
trans
inputstate_post
votereg_post
CC16_414_1
CC16_414_2
CC16_414_3
CC16_414_4
CC16_414_5
CC16_414_6
CC16_417a_1
CC16_417a_2
CC16_417a_3
CC16_417a_4
CC16_417a_5
CC16_421a
CC16_422d
CC16_422c
edloan
CC16_422f
CC16_422e
CC16_426_1
CC16_426_2
CC16_426_3
CC16_426_5
CC16_418a
CC16_426_4
voted
age
CC16_422d_Disagree
CC16_422d_Neutral
CC16_422c_Disagree
CC16_422c_Neutral
CC16_422f_Disagree
CC16_422f_Neutral
CC16_422e_Disagree
CC16_422e_Neutral
CC16_426_1_Increase
CC16_426_1_Maintain
CC16_426_2_

In [91]:
df.groupby(['employ']).size()

employ
Full-time               22367
Homemaker                4012
Other                     999
Part-time                5407
Permanently disabled     3323
Retired                 12236
Student                  1689
Temporarily laid off      304
Unemployed               2562
dtype: int64

In [92]:
df = df.merge(pd.get_dummies(df['employ'], prefix = 'employ', drop_first = True), left_index = True, right_index = True)

In [93]:
df.groupby(['marstat']).size()

marstat
Divorced                 5958
Domestic partnership     2211
Married                 29708
Separated                 787
Single                  11482
Widowed                  2714
dtype: int64

In [94]:
df = df.merge(pd.get_dummies(df['marstat'], prefix = 'mar', drop_first = True), left_index = True, right_index = True)

In [95]:
df.groupby(['pew_religimp']).size()

pew_religimp
Not at all important    11199
Not too important        8216
Somewhat important      13956
Very important          19504
dtype: int64

In [96]:
df = df.merge(pd.get_dummies(df['pew_religimp'], prefix = 'religimp', drop_first = True), left_index = True, right_index = True)

In [97]:
len(df.columns)

149

In [98]:
for col in df.columns:
    print (col, df[col].unique())

tookpost [1]
gender [0 1]
educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Asian' 'Other' 'Hispanic' 'Native American' 'Black'
 'Middle Eastern']
CC16_331_1 [0 1]
CC16_331_2 [1 0]
CC16_331_3 [0 1]
CC16_331_7 [1 0]
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Retired' 'Unemployed' 'Other' 'Student']
milstat_1 [0 1]
milstat_2 [0 1]
milstat_3 [0 1]
milstat_4 [1 0]
CC16_351K [  1.   0.  nan]
faminc [  54999.   64999.    5000.   24999.   34999.  109999.   44999.   14999.
   74999.   89999.  134999.  500000.  150000.  174999.  424999.  224999.
  299999.]
CC16_334a [  1.   0.  nan]
CC16_334b [  1.   0.  nan]
pew_religimp ['Somewhat important' nan 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal' nan]
CC16_334d [  1.   0.  nan]
pew_bornagain [  0.  nan   1.]
CC16_334c [  0.   1.  nan]
CC16

In [99]:
df = df.dropna(axis = 0, how = 'any')

df.to_csv('EDA.csv')
len(df.index)

48130

**Dropping columns**

For machine learning, the text variables need to be removed.

In [100]:
todrop = []
for col in df.columns:
    has_string = False
    for value in df[col].unique():
        if isinstance(value, str):
            has_string = True
    if has_string:
        todrop.append(col)
        
for col in todrop:
    print (col, df[col].unique())
#todrop = ['tookpost', 'educ', 'race', 'employ', 'pew_religimp', 'ideo5', 'marstat', 'religpew', 'ownhome', 'union', 'immstat',
#         '']

educ ['High school graduate' '4-year' 'Some college' 'No HS' '2-year'
 'Post-grad']
race ['White' 'Mixed' 'Other' 'Hispanic' 'Native American' 'Black' 'Asian'
 'Middle Eastern']
employ ['Homemaker' 'Part-time' 'Permanently disabled' 'Full-time'
 'Temporarily laid off' 'Unemployed' 'Retired' 'Other' 'Student']
pew_religimp ['Somewhat important' 'Not at all important' 'Very important'
 'Not too important']
ideo5 ['Moderate or Not sure' 'Liberal' 'Very conservative' 'Conservative'
 'Very liberal']
marstat ['Married' 'Widowed' 'Separated' 'Single' 'Divorced' 'Domestic partnership']
religpew ['Nothing in particular' 'Agnostic' 'Protestant' 'Roman Catholic' 'Atheist'
 'Something else' 'Eastern or Greek Orthodox' 'Jewish' 'Buddhist' 'Mormon'
 'Muslim' 'Hindu']
ownhome ['Own' 'Rent' 'Other']
union ['I am not now, nor have I been, a member of a labor union'
 'Yes, I am currently a member of a labor union'
 'I formerly was a member of a labor union']
immstat ['Third generation' 'First generation

In [101]:
extra_clean = df.drop(todrop, axis = 1)

In [102]:
len(extra_clean.columns)

124

In [103]:
extra_clean.to_csv('cleaned.csv')